In [57]:
import pandas as pd
import numpy as np

In [58]:
data = pd.read_csv("data/indicator_data/BEL-USD.csv")

In [59]:
data.head()

,Date,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2020-09-16,6.171814,6.244687,4.075332,4.608443,58191896,-1.264755e+08,105996915,-0.568734,-9.789113e+07,...,-91.052855,0.000000,6.054850,0.0,-5.408562,-1.081712,-4.326850,-26.741449,-31.117521,-26.741449
1,2020-09-17,4.585522,4.921655,3.020856,3.020856,20733476,-1.472090e+08,85263439,-0.605514,-8.860899e+07,...,-100.000000,0.000000,5.919202,0.0,-12.285064,-3.322383,-8.962681,-34.449530,-42.234981,-51.978676
2,2020-09-18,3.045186,3.854492,2.853754,3.001233,14830439,-1.576683e+08,70433000,-0.611248,-7.599214e+07,...,-97.945980,0.000000,5.790403,0.0,-18.816058,-6.421118,-12.394940,-0.649578,-0.651697,-52.290611
3,2020-09-19,3.019029,5.558504,2.987119,4.594291,74813134,-1.389617e+08,145246134,-0.417606,-4.811017e+07,...,-75.758610,0.000000,5.693580,0.0,-20.204764,-9.177847,-11.026917,53.080119,42.579125,-26.966411
4,2020-09-20,4.639493,5.006308,3.481535,3.685932,21665054,-1.548183e+08,123581080,-0.436818,-4.404866e+07,...,-88.409815,-0.579884,5.598443,0.0,-25.332520,-12.408782,-12.923738,-19.771478,-22.029110,-41.406231


In [60]:
LOOKBACK = 7
FEATURE_NUMBER = 12
complete_data = pd.DataFrame()

columns = np.delete(data.columns, [0, 4])
target = "Close"
new_df = data[columns].sample(n=FEATURE_NUMBER, axis='columns')
new_df["Close"] = data["Close"]
for index, row in new_df.iterrows():
    try:
        tmp_df = new_df.iloc[index:index+LOOKBACK,:]
        tmp_X = tmp_df.iloc[:,:FEATURE_NUMBER]
        tmp_y = new_df.iloc[index+LOOKBACK,-1]
        tmp_X = tmp_X.to_numpy().flatten()
        tmp_X = pd.Series(tmp_X)
        tmp_y = pd.Series(tmp_y)
        tmp_var = tmp_X.append(tmp_y, ignore_index=True)
        tmp_var = tmp_var.values.reshape(1, len(tmp_var.values))
        tmp_var = pd.DataFrame(tmp_var)
        tmp_var.columns = [*tmp_var.columns[:-1], 'Close']
        complete_data = complete_data.append(tmp_var, ignore_index=True)
    except:
        continue

complete_data["Close"] = complete_data["Close"].pct_change()
complete_data = complete_data.dropna(axis=0)
complete_data["Close"] = complete_data["Close"].apply(lambda x: 'Buy' if x > 0 else 'Sell')
complete_data

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,Close
1,5.282438,1979.934908,7.310137,4.639985,20733476.0,6.376607,8.208521,2.369234,6.527316,9.814390,...,3.701346,8073182.0,5.485956,5.793309,2.369234,6.177789,2.087085,127.208995,-4.425185,Buy
2,4.770952,1812.142741,6.943490,4.230297,14830439.0,6.196067,7.215698,2.369234,6.443765,3.667055,...,3.590424,6767329.0,5.434292,5.500014,2.369234,6.175344,2.357731,129.815585,-3.983818,Sell
3,4.692756,1719.413722,6.747258,4.303096,74813134.0,5.787759,7.162830,2.369234,6.443765,8.765136,...,3.488254,5493880.0,5.393244,4.586242,2.369234,6.175344,2.261088,132.677313,-3.575507,Sell
4,4.586951,1648.238014,6.478366,4.200235,21665054.0,5.682018,6.899475,2.369234,6.443765,12.628531,...,3.391567,3654509.0,5.365515,4.128227,2.369234,6.171063,1.919998,136.014169,-3.298367,Sell
5,4.401921,1584.603664,6.293294,4.036133,11724967.0,5.605475,6.515167,2.369234,6.443765,12.861989,...,3.286700,6001124.0,5.316846,3.823448,2.369234,5.976525,1.179098,141.910078,-3.182161,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.855881,29.962741,1.249684,0.986084,11368105.0,0.878168,0.919044,0.898319,1.028763,30.398296,...,0.928663,8271610.0,0.850578,0.901113,0.941399,1.015775,32.789007,23.168370,-0.003573,Buy
582,0.858530,0.806325,1.220060,0.979841,9802025.0,0.867777,0.918410,0.898319,1.028763,32.679544,...,0.913503,7567474.0,0.844496,0.892640,0.964049,1.015463,22.453949,19.233999,-0.006299,Buy
583,0.859323,-24.740271,1.174386,0.971679,7367922.0,0.862581,0.915192,0.898319,1.028763,36.528423,...,0.897103,9727858.0,0.838568,0.883801,0.976277,1.010871,14.824630,19.859844,-0.006912,Sell
584,0.856990,-46.251521,1.100337,0.965459,7358324.0,0.857888,0.909791,0.898319,1.028763,36.025411,...,0.875932,9718821.0,0.829288,0.870650,0.976277,0.988630,14.638839,21.972110,-0.009204,Sell


In [61]:
from sklearn.model_selection import train_test_split

X = complete_data.iloc[:,:-1]
y = complete_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

In [62]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=100)
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [63]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(accuracy_score(y_test, preds))
print(confusion_matrix(y_test, preds))

0.48717948717948717
[[31 29]
 [31 26]]


In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

param_grid = {
    "criterion": ["gini", "entropy"],
    "splitter": ["best", "random"],
    "max_depth": [1, 10, 100, 150, 250, 500]
    # "min_samples_split": [1, 10, 100, 150, 250, 500],
    # "min_samples_leaf": [1, 10, 100, 150, 250, 500],
    # "max_features": ["auto", "sqrt", "log2"],
    }
criterion = ["gini", "entropy"]
splitter = ["best", "random"]
max_depth = [1, 10, 100, 150, 250, 500, 1000, 10000]

best_acc = 0
best_params = []

for p1 in criterion:
    for p2 in splitter:
        for p3 in max_depth:
            model = DecisionTreeClassifier(criterion=p1, splitter=p2, max_depth=p3)
            model.fit(X_train, y_train)
            train_preds = model.predict(X_train)
            test_preds = model.predict(X_test)
            val_preds = model.predict(X_val)
            # print(p1, p2, p3)
            # print("Train acc: ", accuracy_score(y_train, train_preds))
            # print("Test acc: ", accuracy_score(y_test, test_preds))

            if accuracy_score(y_test, test_preds) > best_acc:
                del best_params
                print(p1,p2,p3)
                best_acc = accuracy_score(y_test, test_preds)
                val_acc = accuracy_score(y_val, val_preds)
                print("Test acc: ", best_acc)
                print("Val acc: ", val_acc)
                best_params = [p1, p2, p3]
                print()

print(best_acc)
print(best_params)

gini best 1
Test acc:  0.5213675213675214
Vall acc:  0.49572649572649574

gini best 10
Test acc:  0.5384615384615384
Vall acc:  0.5555555555555556

gini random 150
Test acc:  0.5982905982905983
Vall acc:  0.5555555555555556

0.5982905982905983
['gini', 'random', 150]
